In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

**Proceso:**
- Una calle y un sentido: Cada partícula avanza con probabilidad p solo si esta despejado.

**Queremos:**
- Función que simule TASEP: recibe probabilidad de avanzar, si es secuencial o paralelo, tamaño de la calle, cantidad inicial de particulas y tasa de generación de particulas.

In [3]:
class Particula:

    """
    Clase que representa una partícula en un sistema.
    Atributos:
    - posicion (int): La posición actual de la partícula.
    - bloqueado (bool): Indica si la partícula está bloqueada o no.
    Métodos:
    - __init__(posicion: int, bloqueado: bool): Inicializa una nueva instancia de la clase Particula.
    - avanzar(): Avanza la partícula una posición hacia adelante.
    - __str__(): Devuelve una representación en cadena de la partícula como str.
    """

    def __init__(self, posicion: int, bloqueado: bool) -> None:
        self.posicion = posicion
        self.bloqueado = bloqueado
    
    def avanzar(self) -> None:
        self.posicion += 1

    def __str__(self) -> str:
        return f'Posición: {self.posicion}, Bloqueado: {self.bloqueado}'


In [4]:
class Calle(list):

    """
    Clase que representa una calle en un modelo de simulación TASEP.
    Métodos:
    - __init__(*args): Constructor de la clase.
    - agregar_particula_inicio(): Agrega una partícula al inicio de la calle.
    - update_bloqueo_casilla(i: int): Actualiza el estado de bloqueo de una casilla en la calle.
    - update_bloqueo(): Actualiza el estado de bloqueo de todas las casillas en la calle.
    - update_secuencial(p: float): Actualiza la posición de las partículas en la calle de forma secuencial.
    - update_paralelo(p: float): Actualiza la posición de las partículas en la calle de forma paralela.
    - plot(): Grafica la posición de las partículas en la calle.
    - animate(n: int, p: float, mode: str): Genera una animación de la simulación.
    - __str__(): Retorna una representación en cadena de la calle.
    """

    def __init__(self, *args) -> None:
        super().__init__(*args)

    def agregar_particula_inicio(self) -> None:
        self.insert(0, Particula(0, self[0].posicion == 1))

    def update_bloqueo_casilla(self, i: int) -> None:
        self[i].bloqueado = self[i+1].posicion == self[i].posicion + 1

    def update_bloqueo(self) -> None:
        for i in range(len(self) - 1):
            self.update_bloqueo_casilla(i)
    
    def update_secuencial(self, p: float) -> None:
        for i in range(len(self)-1, -1, -1):
            if not self[i].bloqueado and np.random.rand() < p:
                self[i].avanzar()
                if i != len(self) - 1:
                    self.update_bloqueo_casilla(i)
                if i != 0:
                    self[i-1].bloqueado = False

    def update_paralelo(self, p: float) -> None:
        for i in range(len(self)-1, -1, -1):
            if not self[i].bloqueado:
                if np.random.rand() < p:
                    self[i].avanzar()
        self.update_bloqueo()

    def plot(self) -> None:
        plt.plot([p.posicion for p in self], [0 for _ in self], 'ro')
        plt.show()

    def animate(self, n: int, p: float, mode: str) -> None:
        fig, ax = plt.subplots()
        line, = ax.plot([], [], 'ro')

        def init():
            ax.set_xlim(0, len(self))
            ax.set_ylim(0, 1)
            return line,

        def update(frame):
            if mode == 'secuencial':
                self.update_secuencial(p)
            elif mode == 'paralelo':
                self.update_paralelo(p)
            line.set_data([p.posicion for p in self], [0 for _ in self])
            ax.set_xlim(0, len(self))
            return line,

        ani = animation.FuncAnimation(fig, update, frames=n, init_func=init, blit=True)
        ani.save('animation.gif', writer='pillow')

    def __str__(self) -> str:
        return '\n'.join([str(p) for p in self])